In [6]:
from constants import *
from data_preprocess import *
from data_postprocess import *
from unet import *
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

For Google Colab

In [7]:
DATA_ROOT = 'data/'
PATH_ROOT = ''

### Load Colab Notebook settings

**Link Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH_ROOT = 'gdrive/My Drive/'  #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Try GPU power**

In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=d2a73514576db0602c11882accbcd4d819cec20b1dcc7664a0075c70743694b3
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  | Proc size: 591.9 MB
GPU RAM Free: 16015MB | Used: 265MB | Util   2% | Total 16280MB


In [ ]:
#!kill -9 -1

**1. Split data into validation and training directories** [Local]

In [8]:
create_validation_train_directory(DATA_ROOT+'train/',
                                  'images',
                                  'groundtruth', 1)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/images'

**2. Data augmentation, generators, callbacks**

In [ ]:
data_gen_args = dict(rotation_range=0.15,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

train_generator = dataGenerator(2,
                                PATH_ROOT+DATA_ROOT+'/train',
                                'images_tr',
                                'groundtruth_tr'
                                ,data_gen_args,
                                (SIDE,SIDE))

validation_generator = dataGenerator(2,
                                PATH_ROOT+DATA_ROOT+'/train',
                                'images_te',
                                'groundtruth_te'
                                ,data_gen_args,
                                (SIDE,SIDE))

filepath = "weights.{epoch:02d}-{acc:.2f}.hdf5"
cp_callback = ModelCheckpoint(filepath=filepath, verbose=1, save_weights_only=True, period=1)

**3. Load the model**

In [9]:
model = unet((SIDE,SIDE,3), verbose=True)

















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalizat

**4. Train the model**

In [10]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=2000,
                    epochs=10, 
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps = 700,
                    #validation_freq=1,
                    initial_epoch=0,
                    callbacks=[cp_callback]
                    )



Epoch 1/10
Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 96 images belonging to 1 classes.
Found 96 images belonging to 1 classes.
2000/2000 [==============================] - 366s 183ms/step - loss: 0.2308 - acc: 0.9166 - val_loss: 0.1959 - val_acc: 0.9374

Epoch 00001: saving model to weights.01-0.92.hdf5
Epoch 2/10
2000/2000 [==============================] - 287s 143ms/step - loss: 0.1356 - acc: 0.9587 - val_loss: 0.1733 - val_acc: 0.9420

Epoch 00002: saving model to weights.02-0.96.hdf5
Epoch 3/10
2000/2000 [==============================] - 286s 143ms/step - loss: 0.0985 - acc: 0.9691 - val_loss: 0.1985 - val_acc: 0.9365

Epoch 00003: saving model to weights.03-0.97.hdf5
Epoch 4/10
2000/2000 [==============================] - 287s 143ms/step - loss: 0.0669 - acc: 0.9785 - val_loss: 0.1859 - val_acc: 0.9411

Epoch 00004: saving model to weights.04-0.98.hdf5
Epoch 5/10
2000/2000 [==============================] - 287s 144ms/step - loss: 0.

### Save model to disk

In [10]:
save_model(model, "model_saved/Model_UNET_256_acc99_valid.json")

NameError: name 'model' is not defined

____
____
____
### Predictions (don't run it on Colab)

**Load model** [Optional]

In [3]:
from keras.models import model_from_json
json_file = open('model_saved/Model_UNET_256_acc99_valid.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

**Resize images in a folder** [Optional]

In [ ]:
reshape_img_test("data/tmp/", "data/test/", SIDE)

**Predict on test images and save the results**

In [ ]:
data_test = data_load('data/test/')

test_datagenerator  = ImageDataGenerator()
testGene  = test_datagenerator.flow(data_test, batch_size=1)

model = uneti(input_size = (SIDE,SIDE,3), pretrained_weights=None, verbose=False)

model.load_weights("model_saved/Model_UNET_256_acc99_valid.h5")

results = model.predict(data_test,verbose=1)
saveResult("data/test/", "data/pred_unet_256_99_valida", results, concat=False)
saveResult("data/test/", "data/pred_unet_256_99_validb", results, concat=True)

**Resize image predictions to final size**

In [ ]:
path_from = "data/MEAN/pred_unet_512_98a/"
path_to = path_from

reshape_img_test(path_from, path_to, SIDE_FINAL)

**Load prediction before averaging them**

In [ ]:
original_data = data_load("data/tmp/", RGB_image=True)
IMGS = []
IMGS.append(data_load("data/MEAN/pred_unet_256_97_learning_1e-2a/"))
IMGS.append(data_load("data/MEAN/pred_unet_256_99_valida/"))

means = average_image(IMGS)
print('Size should be (608, 608) and currently is:',means[0].shape)

**Plot predictions**

In [ ]:
f = 6
columns = 4
rows = int(len(IMGS))+1

fig=plt.figure(figsize=(10, 2*rows))
img_to_plot = IMGS[0][f:f+4]
for i in range(1, int(len(IMGS))):
    img_to_plot = np.append(img_to_plot,IMGS[i][f:f+4], axis=0)
img_to_plot = np.append(img_to_plot, means[f:f+4], axis=0)
for i in range(1, columns*rows +1):
    img = img_to_plot[i-1]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
patched_imgs, vs = patched_imgs_and_vs(means)

fig=plt.figure(figsize=(15, 10))
f = 6
columns = 4
rows = 3
img_to_plot0 = means[f:f+4]
img_to_plot1 =np.append(img_to_plot0, patched_imgs[f:f+4], axis=0)
for i in range(1, columns*rows +1):
    if(i<=4):
        img = original_data[f+i-1]
    else:
        img = img_to_plot1[i-5]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

**Create submissions**

In [ ]:
create_csv_submission(patched_imgs, vs, 'submissions/test_sub.csv')